In [1]:
# setup
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import zipfile
from tqdm import tqdm

# Random generator
rng1 = np.random.RandomState(seed=123)

In [2]:
train_df_ori = pd.read_csv('titanic_data/train.csv')
test_df_ori = pd.read_csv('titanic_data/test.csv')

train_df = train_df_ori.copy()
test_df = test_df_ori.copy()

In [3]:
display(train_df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
irrelevant_feats = ['PassengerId', 'Name']
train_df.drop(irrelevant_feats, axis=1, inplace=True)
test_df.drop(irrelevant_feats, axis=1, inplace=True)

In [5]:
missing_too_many = ['Cabin', 'Ticket']
train_df.drop(missing_too_many, axis=1, inplace=True)
test_df.drop(missing_too_many, axis=1, inplace=True)

In [6]:
for column in train_df.columns[1:]:
    train_df.dropna(subset=[column], axis=0, inplace=True)
    test_df.dropna(subset=[column], axis=0, inplace=True)

In [7]:
display(train_df.head())

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
def dataframe_onehotencoder(encoder, df, columns_names, fit_transform=True):
    '''
    This is a helper function to apply sklearn onehotencoder function
    on multiple columns of a data frame.

    Args:
        encoder: initialized OneHotEncoder
        df: Pandas dataframe with data
        columns_names: names of the columns in dataframe to encode
        fit_transform: True when the encoder is seeing the categorical data for the first time; False otherwise.
    
    Returns:
        dataframe: with one-hot-encoded columns
    '''
    
    # One-hot-encode the columns in column names
    if(fit_transform):
        data = encoder.fit_transform(df[columns_names])
    else:
        data = encoder.transform(df[columns_names])

    # Convert data to a dataframe
    df2 = pd.DataFrame(data, columns=encoder.get_feature_names_out(columns_names), index=df.index)
    
    # Recreate original dataframe
    df3 = pd.concat([df.drop(columns_names, axis=1), df2], axis=1)
    
    # Return
    return df3 

In [9]:
df_OHE = pd.get_dummies(train_df, drop_first=True)
df_OHE_test = pd.get_dummies(test_df, drop_first=True)

In [10]:
def dataframe_normalization(transformer, df, fit_transform=True):
    '''
    This is a helper function to normalize the dataset features using the ColumnTransformer class.

    Args:
        transformer: initialized ColumnTransformer
        df: Pandas dataframe with the data
        fit_transform: True when the transformer is seeing the data for the first time; False otherwise.
    
    Returns:
        dataframe: with specific normalized columns
    '''
    
    # Copy data
    df2 = df.copy()

    if(fit_transform):
        data = transformer.fit_transform(df2)
    else:
        data = transformer.transform(df2)

    # Add normalized data to dataframe
    for index, column in enumerate(transformer.get_feature_names_out()):
        df2[column] = data[:,index]
        
    # Return
    return df2 

In [11]:
y = df_OHE['Survived'].values
X = df_OHE.iloc[:,1:].values

In [12]:
X_test = df_OHE_test.iloc[:,1:].values

In [13]:
def sigmoid(z):
    z = z.to_numpy(dtype=np.float64)
    z = np.asarray(z, dtype=np.float64)
    return (1/(1 + np.exp(-z)))

In [14]:
def initialize_params(m_features, seed=None):
    rng = np.random.default_rng(seed)
    w = rng.normal(loc=0.0, scale=1.0, size=m_features)
    b = np.float64(0.0)
    return w, b

In [1]:
def forward(X,w,b):
    z = X.T.dot(w) + b
    y_hat = sigmoid(z)
    return z, y_hat

In [2]:
def compute_cost(y,y_hat):
    return (y * np.log(y_hat) + (1-y) * np.log(1-y_hat))

In [ ]:
def compute_gradients(X, y, y_hat):
    